# Introduction



Hello, I'm going to preciton with Logistic regression KNN Classification SVM Decision Tree,adaboost, voting classifier and Random Forest. I will use Heart Disease UCI dataset. If you wonder anything about dateset,you can read here. (https://www.kaggle.com/ronitf/heart-disease-uci)

# Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap #Sonuç görselleştirme
#Libraries for ML
from sklearn.preprocessing import RobustScaler #Standardizasyon için
from sklearn.model_selection import train_test_split, GridSearchCV #GridSearchCV: KNN ile ilgili en iyi parametreleri belirlemek
from sklearn.metrics import accuracy_score ,confusion_matrix #Sonuç değerlendirme
from sklearn.neighbors import KNeighborsClassifier,  LocalOutlierFactor #Trainin algoritması ve NCA ve Outlier değerler için
from sklearn.decomposition import PCA #PCA için
from matplotlib.colors import ListedColormap
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier

#Others
import warnings
warnings.filterwarnings('ignore') #Uyarıları kapatmak



# ML Classification

----Content

1-Import Dataset

2-Investigation Dataset

3-Visualizaition Dataset

4-ML Algoritms importation and predicition

5-Visualizaition of results

6-Conclusion

## Data Reading

In [ ]:
#read data
data = pd.read_csv("../input/heart.csv")

## Exploratory Data Analysis

In [ ]:
data.info()

In [ ]:
print(len(data)) #data uzunluğu 
print('Data Shape',data.shape)

In [ ]:
describe = data.describe() #Statsitiksel değerlerin tespiti
print(describe.T)

In [ ]:
data.tail()


## Target Visualization

* The "goal" field refers to the presence of heart disease in the patient. Target = 1 => presence of heart disease Target = 0 => no of heart disease

In [ ]:
sns.countplot(data['target']) 
print(data.target.value_counts())

### Correlation Matrix of Features

In [ ]:
corr_data = data.corr() # Datada bulunan numerik değerler arasındaki korelasyona bakar.
sns.clustermap(corr_data,annot= True,fmt = '.2f')
#annot grafik üstündeki sayısal değerleri gösteririken fmt ise virgülden sonra kaç basamak gösterilecek bunu belirler.
plt.title('Correlation Between Features')
plt.show();

### Correlation Matrix of Features (with Threshold)

In [ ]:
threshold = 0.28 #Bu eşik değeri ile sadece bu değerin üstündeki korelasyonları değerlendireceğiz yeni grafikte

filtre = np.abs(corr_data['target']) > threshold # Burada corelasyon değerleri negatifde olacağından mutlak değerini alıp tresholddan büyükları filtreledik.
corr_feature = corr_data.columns[filtre].tolist()
#Bu değişkene ise korelasyon matrisi sütunlarına filtrenin uygulanmasıyla çıkan çıktıları listeye çevirip atadık.

sns.clustermap(data[corr_feature].corr(),annot= True,fmt = '.2f')
#Buradan eşik değerine uygun olarak elde edilen featureların uygulandığı corr matr. oluşturduk.

plt.title('Correlation Between Features with threshold 0.28')
plt.show();

### Box Plotting before Standardization

In [ ]:
#Box p. öncesi bir melted işlemi gerekitor.
data_melted = pd.melt(data,id_vars='target',
                      var_name='Features',
                      value_name='Value')

plt.figure()
sns.boxplot(x='Features',y='Value',hue='target',data=data_melted) #Featureslar target'a göre ayrıldı.
plt.xticks(rotation=75) #Feature isimleri 90 derece dik görülecek.
plt.show()

### Pair Plotting before Standardization

In [ ]:

sns.pairplot(data[corr_feature],diag_kind='kde',markers='+',hue='target')
plt.show()


### Pre-processing before standardization

In [ ]:
x = data.drop(['target'],axis=1) 
y = data.target
columns = x.columns.tolist() # Featureların isimlerini bir listede topladık.

## Standardization & Train - Tespt Split 

In [ ]:
test_size = 0.2
x = RobustScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=test_size,random_state=42)

### Box plotting after standardization

In [ ]:

x_train_df = pd.DataFrame(x_train,columns=columns)
x_train_df_describe = x_train_df.describe()
x_train_df['target'] = y_train

#Box p. öncesi bir melted işlemi gerekitor.
data_melted = pd.melt(x_train_df,id_vars='target',
                      var_name='Features',
                      value_name='Value')

plt.figure()
sns.boxplot(x='Features',y='Value',hue='target',data=data_melted) #Featureslar target'a göre ayrıldı.
plt.xticks(rotation=75) #Feature isimleri 90 derece dik görülecek.
plt.show()

### Pair plotting after standardization

In [ ]:

sns.pairplot(x_train_df[corr_feature],diag_kind='kde',markers='+',hue='target')
plt.show()

## K-nearest neighbors classifier 

In [ ]:
random_state = 42
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(x_train, y_train) #Calculation (In the supervise learning this section = training)
y_predict = knn.predict(x_test) #Test Section
cm = confusion_matrix(y_test, y_predict) #Plooting
acc = accuracy_score(y_test, y_predict) #Accuracy Score
score = knn.score(x_test, y_test) #acc ile burasının sonucu aynı çıkacak. Doğrulama amaçlı yapılıyor.

print("Score:",score)
print("CM:",cm)
print("Basic KNN Acc:",acc)

### Choose Best Parameters with GridSearchCrossValidation 

In [ ]:
def KNN_best_parameters(x_train,x_test,y_train,y_test):
    
    k_range = list(range(1,51)) #En uygun k değeri buluma
    weight_options = ['uniform','distance'] #En uygun weighti buluma
    #manhattan_distance = 1
    #euclidean_distance = 2
    distance_options = [1,2] #En uygun distance type buluma
    print()
    param_grid = dict(n_neighbors=k_range,weights=weight_options,p=distance_options) #Aranacak parametreleri bir sözlükte topladık.

    knn =KNeighborsClassifier() #Parametrelerin deneneceği knn oluşturuldu.
    grid = GridSearchCV(knn,param_grid,cv=10,scoring='accuracy') #Parametrelerin aranması için method
    grid.fit(x_train, y_train) #fitting ile best parm. elde edildi
    
    print('Best training score: {} with parametres: {}'.format(grid.best_score_,grid.best_params_))
    print()
    
    knn = KNeighborsClassifier(**grid.best_params_) #Test setinde deneme işlemi için
    knn.fit(x_train, y_train)
    
    y_predict_test = knn.predict(x_test)
    y_predict_train = knn.predict(x_train)

    cm_test = confusion_matrix(y_test,y_predict_test)
    cm_train = confusion_matrix(y_train,y_predict_train)

    acc_test = accuracy_score(y_test,y_predict_test)  
    acc_train = accuracy_score(y_train,y_predict_train)

    print('Test Score: {}, Train Score: {}'.format(acc_test,acc_train))
    print()
    print('CM Test:',cm_test)
    print('CM Train:',cm_train)
    
    return grid

grid_knn_bestparam = KNN_best_parameters(x_train,x_test,y_train,y_test)

In [ ]:
knn_score=grid_knn_bestparam.score(x_test,y_test)

### Principal component analysis

In [ ]:
scaler = RobustScaler()
x_scaled = scaler.fit_transform(x) #x verisi bölünmeden tam bir şekilde PCA için scale edildi. 

pca = PCA(n_components=2) #2 componentli bir PCA oluşturduk.
pca.fit(x_scaled)
x_reduce_pca = pca.transform(x_scaled) #2feature'a yani boyuta düşürülmüş x 
pca_data =pd.DataFrame(x_reduce_pca,columns=['p1','p2']) #reduce datadan incelemek için bir dataframe oluşturuldu
pca_data['target'] = y #buna target eklendi. Görselleştirmek için gerekli.

sns.scatterplot(x='p1',y='p2',hue='target',data=pca_data) # targeta göre renklendirilmiş grafik
plt.title('PCA: P1 Vs P2')

### KNN fit towards PCA outputs

In [ ]:
x_train_pca, x_test_pca, y_train_pca, y_test_pca = train_test_split(x_reduce_pca,y,test_size=test_size,random_state=42)

grid_pca = KNN_best_parameters(x_train_pca, x_test_pca, y_train_pca, y_test_pca)
#en iyi parametreleri elde ettiğimiz metodu PCA için çalıştırıyorum.

### Wrong classification in PCA_KNN

In [ ]:
cmap_light = ListedColormap(['orange',  'cornflowerblue'])
cmap_bold = ListedColormap(['darkorange', 'darkblue'])

h = .05 # step size in the mesh
X = x_reduce_pca
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z = grid_pca.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
            edgecolor='k', s=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("%i-Class classification (k = %i, weights = '%s')"
          % (len(np.unique(y)),grid_pca.best_estimator_.n_neighbors, grid_pca.best_estimator_.weights))


### Support vector classifier

In [ ]:
svc = SVC()
svc.fit(x_train,y_train)  #learning 
#SVM Test 

SVMscore_test = svc.score(x_test,y_test)
SVMscore_train = svc.score(x_train,y_train)
print ("SVM Test Accuracy:", svc.score(x_test,y_test))

print ("SVM Train Accuracy:", svc.score(x_train,y_train))

### Confusion Matrix

In [ ]:
yprediciton2= svc.predict(x_test)
ytrue = y_test

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(ytrue,yprediciton2)

#CM visualization

import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(CM,annot = True, linewidths=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("Prediction(Ypred)")
plt.ylabel("Ytrue")
plt.show()

### Decision Tree Classifier

In [ ]:
dt = DecisionTreeClassifier(random_state=random_state,max_depth=4)

dt.fit(x_train,y_train) #learning
#prediciton
DTCscore_test = dt.score(x_test,y_test)
DTCscore_train = dt.score(x_train,y_train)
print("Decision Tree Test Score: ",dt.score(x_test,y_test))
print("Decision Tree Train Score: ",dt.score(x_train,y_train))

### Confusion Matrix

In [ ]:
yprediciton2= dt.predict(x_test)
ytrue = y_test

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(ytrue,yprediciton2)

#CM visualization

import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(CM,annot = True, linewidths=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("Prediction(Ypred)")
plt.ylabel("Ytrue")
plt.show()

### Optimum N Estimator Value for Random Forest

In [ ]:
scores = []
for each in range(1,50):
    rf = RandomForestClassifier(n_estimators = each,random_state=random_state,max_depth=3)
    rf.fit(x_train,y_train)
    scores.append(rf.score(x_test,y_test))
    
plt.figure(1, figsize=(10, 5))
plt.plot(range(1,50),scores,color="black",linewidth=2)
plt.title("Optimum N Estimator Value")
plt.xlabel("N Estimators")
plt.ylabel("Score(Accuracy)")
plt.grid(True)
plt.show()

### Random Forest Classifier

In [ ]:
rf= RandomForestClassifier(n_estimators = 19, random_state=random_state,max_depth=3) #n_estimator = DT
rf.fit(x_train,y_train) # learning
RFCscore=rf.score(x_test,y_test)

print("Random Forest Test Score: ",rf.score(x_test,y_test))
print("Random Forest Train Score: ",rf.score(x_train,y_train))

### Confusion Matrix

In [ ]:
yprediciton2= rf.predict(x_test)
ytrue = y_test

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(ytrue,yprediciton2)

#CM visualization

import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(CM,annot = True, linewidths=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("Prediction(Ypred)")
plt.ylabel("Ytrue")
plt.show()

### Optimum N Estimator Value for Adaboost

In [ ]:
scores = []
for each in range(1,10):
    ada = AdaBoostClassifier(n_estimators = each,random_state=random_state)
    ada.fit(x_train,y_train)
    scores.append(ada.score(x_test,y_test))
    
plt.figure(1, figsize=(10, 5))
plt.plot(range(1,10),scores,color="black",linewidth=2)
plt.title("Optimum N Estimator Value")
plt.xlabel("N Estimators")
plt.ylabel("Score(Accuracy)")
plt.grid(True)
plt.show()

### AdaBoost Classifier

In [ ]:
ada = AdaBoostClassifier(base_estimator=dt,n_estimators=3,random_state=random_state,learning_rate=0.001)
ada.fit(x_train,y_train) # learning
Adascore=ada.score(x_test,y_test)

print("AdaBoost Test Score: ",ada.score(x_test,y_test))
print("AdaBoost Train Score: ",ada.score(x_train,y_train))

### Confusion Matrix

In [ ]:
yprediciton2= ada.predict(x_test)
ytrue = y_test

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(ytrue,yprediciton2)

#CM visualization

import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(CM,annot = True, linewidths=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("Prediction(Ypred)")
plt.ylabel("Ytrue")
plt.show()

### Logistic Regression

In [ ]:
#LR with sklearn
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(x_train,y_train)
print("LR Test Accuracy {}".format(LR.score(x_test,y_test)))
print("LR Train Accuracy {}".format(LR.score(x_train,y_train)))
LRscore =LR.score(x_test,y_test)

### Confusion Matrix

In [ ]:
yprediciton2= LR.predict(x_test)
ytrue = y_test

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(ytrue,yprediciton2)

#CM visualization

import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(CM,annot = True, linewidths=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("Prediction(Ypred)")
plt.ylabel("Ytrue")
plt.show()

### Voting Classifiers

In [ ]:
v1 = VotingClassifier(estimators = [('svc',svc),('knn',knn),('dt',dt),('rf',rf),('ada',ada)])
v1.fit(x_train,y_train) # learning
vooting1=v1.score(x_test,y_test)

print("Voting C. Test Score: ",v1.score(x_test,y_test))
print("Voting C. Train Score: ",v1.score(x_train,y_train))

In [ ]:
v2= VotingClassifier(estimators = [('svc',svc),('knn',knn),('rf',rf)])
v2.fit(x_train,y_train) # learning
vooting2=v2.score(x_test,y_test)

print("Voting C. Test Score: ",v2.score(x_test,y_test))
print("Voting C. Train Score: ",v2.score(x_train,y_train))

## Evalutaion of Results

In [ ]:
scores=[SVMscore_test,DTCscore_test,knn_score,RFCscore,Adascore,LRscore,vooting1,vooting2]
AlgorthmsName=["SVM","Decision Tree","K-NN","Random Forest","Adaboost","Logistic Regression", 'Voting Classifier -1-','Voting Classifier-2-']

#create traces

trace1 = go.Scatter(
    x = AlgorthmsName,
    y= scores,
    name='Algortms Name',
    marker =dict(color='rgba(0,255,0,0.5)',
               line =dict(color='rgb(0,0,0)',width=2)),
                text=AlgorthmsName
)
data = [trace1]

layout = go.Layout(barmode = "group",
                  xaxis= dict(title= 'ML Algorithms',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Prediction Scores',ticklen= 5,zeroline= False))
fig = go.Figure(data = data, layout = layout)
iplot(fig)

# Conclusion

1- Thank you for investigation my kernel.

2- I compared ML algorithms with Heart Disease Dataset.

3- I found optimum value by aid of for loop.

4- Finally, I obtained the same score values.

5- I expect your opinion and criticism.
# If you like this kernel, Please Upvote :) Thanks

<img src="https://media.giphy.com/media/1oF1KAEYvmXBMo6uTS/giphy.gif" width="500px">